In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# Step 1: Prepare your dataset
image_dir = 'C:/DERM-AI PROJECT/Disease_Dataset_5_classes/train'
class_names = os.listdir(image_dir)
num_classes = len(class_names)

# Step 2: Load and preprocess the dataset
images = []
labels = []
for class_name in class_names:
    class_dir = os.path.join(image_dir, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = cv2.imread(os.path.join(class_dir, filename))
            img = cv2.resize(img, (64, 64))  # Resize images to a fixed size
            img = img / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(class_names.index(class_name))

X = np.array(images)
y = np.array(labels)

# Step 3: Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Apply image augmentation
# Data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=30,          # Randomly rotate images by 30 degrees
    width_shift_range=0.2,      # Shift images horizontally by 20%
    height_shift_range=0.2,     # Shift images vertically by 20%
    shear_range=0.2,            # Shear transformation
    zoom_range=0.2,             # Random zoom
    horizontal_flip=True,       # Flip images horizontally
    fill_mode='nearest'         # Fill missing pixels with nearest values
)

# Fit the augmentation generator to the training dataset
datagen.fit(X_train)

# Step 5: Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Step 6: Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 7: Train the model using the augmented data
# Use the datagen.flow function to train with augmented images
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=100,
          validation_data=(X_test, y_test))

# Step 8: Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

# Save the model
model.save('trained_model_with_augmentation.h5')


c:\Users\kadam\Downloads\Derm-Ai (2) (1)\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\kadam\Downloads\Derm-Ai (2) (1)\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.4383 - loss: 1.2389 - val_accuracy: 0.6340 - val_loss: 0.8451
Epoch 2/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.6562 - loss: 0.8594 - val_accuracy: 0.5237 - val_loss: 0.9163
Epoch 3/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.6140 - loss: 0.8595 - val_accuracy: 0.6937 - val_loss: 0.6722
Epoch 4/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.6799 - loss: 0.7920 - val_accuracy: 0.7014 - val_loss: 0.6693
Epoch 5/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.6940 - loss: 0.7566 - val_accuracy: 0.7060 - val_loss: 0.6681
Epoch 6/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.6692 - loss: 0.7542 - val_accuracy: 0.6983 - val_loss: 0.6300
Epoch 7/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.7037 - loss: 0.7150 - val_accuracy: 0.7029 - val_loss: 0.6065
Epoch 8/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.7130 - loss: 0.6749 - val_accuracy: 0.

In [1]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers, callbacks
import os

# Paths for training and validation datasets
train_dir = 'C:/DERM-AI PROJECT/Disease_Dataset_5_classes/train'


# Step 1: Create data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)  # Only rescale for validation

# Step 2: Load images in batches directly from disk
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Match input size for pre-trained models
    batch_size=32,
    class_mode='sparse'
)



# Step 3: Use Transfer Learning with EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base layers initially

# Add custom layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Step 4: Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Add callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-6)
checkpoint = callbacks.ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)

# Step 6: Train the model
history = model.fit(
    train_generator,
    epochs=50,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Step 7: Unfreeze base model for fine-tuning
base_model.trainable = True
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    epochs=20,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Step 8: Save the final model
model.save('final_model_with_transfer_learning.keras')


Found 3265 images belonging to 5 classes.


c:\Users\kadam\Downloads\Derm-Ai (2) (1)\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 89s 793ms/step - accuracy: 0.2263 - loss: 2.7486 - learning_rate: 0.0010
Epoch 2/50


c:\Users\kadam\Downloads\Derm-Ai (2) (1)\.venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
c:\Users\kadam\Downloads\Derm-Ai (2) (1)\.venv\Lib\site-packages\keras\src\callbacks\callback_list.py:151: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
c:\Users\kadam\Downloads\Derm-Ai (2) (1)\.venv\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


103/103 ━━━━━━━━━━━━━━━━━━━━ 84s 814ms/step - accuracy: 0.3817 - loss: 1.9469 - learning_rate: 0.0010
Epoch 3/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 83s 801ms/step - accuracy: 0.3863 - loss: 1.8113 - learning_rate: 0.0010
Epoch 4/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.4132 - loss: 1.7014 - learning_rate: 0.0010
Epoch 5/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 82s 795ms/step - accuracy: 0.4239 - loss: 1.6504 - learning_rate: 0.0010
Epoch 6/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 82s 799ms/step - accuracy: 0.4150 - loss: 1.5929 - learning_rate: 0.0010
Epoch 7/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 83s 801ms/step - accuracy: 0.4322 - loss: 1.5663 - learning_rate: 0.0010
Epoch 8/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 81s 790ms/step - accuracy: 0.4256 - loss: 1.5260 - learning_rate: 0.0010
Epoch 9/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 82s 794ms/step - accuracy: 0.4081 - loss: 1.5105 - learning_rate: 0.0010
Epoch 10/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.4292 - loss: 1.4777 - learning_rate: 0.00

In [3]:
!pip install scikit-learn


   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 493.7 kB/s eta 0:00:22
   -- ------------------------------------- 0.8/11.0 MB 907.1 kB/s eta 0:00:12
   -- ------------------------------------- 0.8/11.0 MB 907.1 kB/s eta 0:00:12
   --- ------------------------------------ 1.0/11.0 MB 719.5 kB/s eta 0:00:14
   --- ------------------------------------ 1.0/11.0 MB 719.5 kB/s eta 0:00:14
   ------ --------------------------------- 1.8/11.0 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.0 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.0 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.0 MB 1.1 MB/s eta 0:00:09
   -------- ------------------------------- 2.4/11.0 MB 952.0 kB/s eta 0:00:10
   -----